## Welcome to BQuant

<img style="float:left;" src="../images/workflow.png" width="500" height="500" />

In [1]:
pwd

'C:\\Users\\MitchellDavid\\AppData\\Local\\bipy\\6316781\\projects\\0000000000000000000000000000000006316781\\Demo Code'

In [1]:
# Import the BQL library
import bql

# Instantiate an object to interface with the BQL service
bq = bql.Service()

# BQL String Interface Basics

This notebook provides an introduction to the string interface. We'll start with a simple example and build on it to learn how to construct more complex requests.

<a href = "#A">Example A: Requesting a Single Data Item</a>  
<a href = "#B">Example B: Requesting Multiple Data Items</a>  
<a href = "#C">Example C: Requesting Time Series and Point-In-Time Data</a>  
<a href = "#D">Example D: Requesting Data for Multiple Securities</a>  
<a href = "#E">Options Examples</a>  
<a href = "#F">ETF Examples</a>  
<a href = "#H">Additional (Portfolio interop, technicals)</a>  
<a href = "#J">Cross Asset</a>  
<br>

## Example A: Requesting a Single Data Item <a name="A"></a>
In this example, we'll create and execute the simplest possible request: a single data item for a single security. We'll build on this request in later examples. For now, let's request the last price for AAPL US Equity. To do this, we'll define a variable for the request and pass it to the BQL execute function.

- **Request syntax**:  
        `get(data fields)for([security or universe])`
- **BQL execute function**:  
        `bq.execute(request)`

_Note: For more information on finding data fields that can be queried with BQL, see <a href = "#related"> Related Resources</a> ._


### Basic Last Price Request

In [2]:
# Define the request string 
request = "get(PX_LAST)for(['ABX CN Equity'])"

# Execute the request
response = bq.execute(request)

# Display the response in a DataFrame
response[0].df()

,DATE,CURRENCY,PX_LAST
ID,,,
ABX CN Equity,2020-08-28,CAD,38.59


<div style="text-align: right"><a href="#BQL-String-Interface-Basics">Return to Top ↑</a></div>

## Example B: Requesting Multiple Data Items <a name="B"></a>
Now that we've seen how to request a single data item, let's expand the previous example to request additional data items. We'll again use AAPL US Equity, but this time we'll request the last price, high price, and low price.

### Example Code

In [3]:
# Define the request string
request = """
get(PX_LAST, PX_HIGH, PX_LOW)
for(['AAPL US Equity'])
"""

# Execute the request
response = bq.execute(request)

# Display the response in a DataFrame
# Use the combined_df function to display the three returned values in a single DataFrame 
bql.combined_df(response)

# Note that you can also display a single data item from the response 
# by running response[0].df(), response[1].df(), or response[2].df(),


,DATE,CURRENCY,PX_LAST,PX_HIGH,PX_LOW
ID,,,,,
AAPL US Equity,2020-08-28,USD,502.450012,505.040009,499.429993


In [4]:
#Define utility BQL function to 
def query_bql(s):
    resp = bq.execute(s)
    df = bql.combined_df(resp)
    return df

In [5]:
request = """
get(PX_LAST, PX_HIGH, PX_LOW)
for(['F US Equity'])
"""

In [6]:
df_response = query_bql(request)
df_response

,DATE,CURRENCY,PX_LAST,PX_HIGH,PX_LOW
ID,,,,,
F US Equity,2020-08-28,USD,6.8871,6.95,6.86


<div style="text-align: right"><a href="#BQL-String-Interface-Basics">Return to Top ↑</a></div>

## Example C: Requesting Time Series and Point-In-Time Data <a name="C"></a>
In the examples above, we didn't specify a data date so BQL returned data for today's date by default. If we want to return data for a specific date or date range, we can pass start and end values as arguments to the BQL range function. Note that the dates are associated with the data fields, not the request.

_Note: To request point-in-time data (for a single date), pass the same value for the start and end parameters._


### Example Code

In [7]:
# Define the request string, specifying the date range for each field
# Since we're using the same range for all of the fields, we can use
# a BQL let clause to define a local variable 
request = """
let(
    #date_range = RANGE(2019-06-05, 2020-02-28);
    )
get(
    PX_LAST(dates=#date_range) as #PX_LAST,
    PX_HIGH(dates=#date_range), 
    PX_LOW(dates=#date_range)
    )
for(
    ['JNJ US Equity', 'F US Equity']
    )
"""

# Execute the request
response = bq.execute(request)

# Display the response in a DataFrame
# Use the combined_df function to display 
# the three returned values in a single DataFrame 
df =bql.combined_df(response)
df.sample(n=5)

,DATE,CURRENCY,#PX_LAST,PX_HIGH(dates=#date_range),PX_LOW(dates=#date_range)
ID,,,,,
F US Equity,2019-09-01,USD,NaN,NaN,NaN
F US Equity,2020-01-23,USD,9.14,9.16,8.93
F US Equity,2019-09-11,USD,9.42,9.43,9.29
F US Equity,2019-09-22,USD,NaN,NaN,NaN
F US Equity,2019-11-13,USD,8.81,9.00,8.80


In [8]:
df.shape

(538, 5)

In [9]:
df2 = df.reset_index()
df2.head()

,ID,DATE,CURRENCY,#PX_LAST,PX_HIGH(dates=#date_range),PX_LOW(dates=#date_range)
0,JNJ US Equity,2019-06-05,USD,134.38,135.13,133.86
1,JNJ US Equity,2019-06-06,USD,136.68,137.66,134.59
2,JNJ US Equity,2019-06-07,USD,138.55,139.59,137.14
3,JNJ US Equity,2019-06-08,USD,NaN,NaN,NaN
4,JNJ US Equity,2019-06-09,USD,NaN,NaN,NaN


In [10]:
df3 = df2.pivot(index='DATE', columns='ID',values = '#PX_LAST')
df3.tail(n=10)

ID,F US Equity,JNJ US Equity
DATE,,
2020-02-19,8.00,148.94
2020-02-20,8.03,148.37
2020-02-21,7.89,149.93
2020-02-22,NaN,NaN
2020-02-23,NaN,NaN
2020-02-24,7.57,145.91
2020-02-25,7.23,144.65
2020-02-26,7.21,143.68
2020-02-27,6.97,139.12


In [11]:
df3.dropna().tail(n=10)

ID,F US Equity,JNJ US Equity
DATE,,
2020-02-14,8.10,150.13
2020-02-18,8.06,149.14
2020-02-19,8.00,148.94
2020-02-20,8.03,148.37
2020-02-21,7.89,149.93
2020-02-24,7.57,145.91
2020-02-25,7.23,144.65
2020-02-26,7.21,143.68
2020-02-27,6.97,139.12


### One-Line Interactive Chart using `bqviz`

In [12]:
import bqviz as bqv

In [13]:
bqv.LinePlot(df3).set_style().show()


GridBox(children=(Figure(animation_duration=500, axes=[Axis(color='white', grid_color='#3c3c3c', grid_lines='d…

<div style="text-align: right"><a href="#BQL-String-Interface-Basics">Return to Top ↑</a></div>

## Example D: Requesting Data for Multiple Securities <a name="D"></a>
Now that we know how to pull multiple data items across a range of dates for a single security, let's look at how we can request data for multiple securities by passing a list as part of the request string.

### Example Code

In [14]:
# Define the request string, specifying the date range for each field
# Since we're using the same range for all of the fields, we can use
# a BQL let clause to define a local variable 
request = """
let(
    #date_range = RANGE(2019-06-05, 2020-02-28);
    )
get(
    PX_LAST(dates=#date_range) as #PX_LAST,
    PX_HIGH(dates=#date_range) as #PX_HIGH, 
    PX_LOW(dates=#date_range) as #PX_LOW
    )
for(
    ['NFLX US Equity', 'AMZN US Equity', 'TSLA US Equity']
    )
"""

# Execute the request
response = bq.execute(request)

# Display the response in a DataFrame
# Use the combined_df function to display 
# the three returned values in a single DataFrame
# To verify the output, use tail(3) to show the last three rows of the response
bql.combined_df(response).sample(n=15)

,DATE,CURRENCY,#PX_LAST,#PX_HIGH,#PX_LOW
ID,,,,,
TSLA US Equity,2020-01-20,USD,NaN,NaN,NaN
NFLX US Equity,2019-10-12,USD,NaN,NaN,NaN
AMZN US Equity,2019-09-21,USD,NaN,NaN,NaN
NFLX US Equity,2019-07-11,USD,379.50,384.54,378.800
TSLA US Equity,2019-09-16,USD,242.81,247.43,241.170
AMZN US Equity,2019-09-11,USD,1822.99,1833.42,1809.080
NFLX US Equity,2020-01-08,USD,339.26,342.70,331.050
TSLA US Equity,2019-10-31,USD,314.92,319.00,313.000
TSLA US Equity,2020-01-12,USD,NaN,NaN,NaN


<div style="text-align: right"><a href="#BQL-String-Interface-Basics">Return to Top ↑</a></div>

## Single Stock Option Open Interest Analysis <a name="E"></a>
#### <font color=orange> Expiries with highest Open Interest </font>

In [ ]:
# Define the request string, specifying the date range for each field
# Since we're using the same range for all of the fields, we can use
# a BQL let clause to define a local variable 
request2 = """
get(
      sum(group(znav(
            OPEN_INT().value), 
            EXPIRE_DT)
        ) as #OPEN_INTEREST
    )
for(
    options(['TSLA US EQUITY'])
    )														
"""

# Execute the request
response2 = bq.execute(request2)

# Display the response in a DataFrame
# Use the combined_df function to display 
# the three returned values in a single DataFrame
# To verify the output, use tail(3) to show the last three rows of the response
#bql.combined_df(response2).tail(3)
newdf = bql.combined_df(response2)
newdf = newdf.drop(['ORIG_IDS'], axis=1)
newdf

## Volatility Smile
#### <font color=orange> Implied Volatility Curve filtered by Open Interest </font>

In [ ]:
# Define the request string, specifying the date range for each field
request = """
get(groupsort(STRIKE_PX, order=ASC), OPEN_INT, IVOL) 
for(
    filter(
           options(['MSFT US Equity']), 
           PUT_CALL==IF(PCT_MONEYNESS<100,PUT,CALL)   AND 
           OPEN_INT > 10 AND 
           YEAR(EXPIRE_DT) ==2020 AND 
           MONTH(EXPIRE_DT)== 3 AND IVOL>0
         ) 
    )
														
"""

# Execute the request
response3 = bq.execute(request)

In [ ]:
# Display the response in a DataFrame
# Use the combined_df function to display 
newdf = bql.combined_df(response3)
newdf.sample(n=20)

## Vega Weighting
#### Average Vega weighted by Open Interest for monthly options on a list

In [ ]:
# Define the request string, specifying the date range for each field
request = """
get(
    wavg(
               group(znav(VEGA),UNDL_TICKER),  
               group(OPEN_INT,UNDL_TICKER) 
    ) as #AVGVEGA_OI
)  
for(
    filter(
        options(
            ['TSLA US EQUITY','CRM US EQUITY','IBM US EQUITY',
            'GOOG US EQUITY', 'MSFT US EQUITY', 'ESL US EQUITY']
            ), 
        EXPIRATION_PERIODICITY==Monthly
        )
    )											
"""

# Execute the request
response4 = bq.execute(request)

# Display the response in a DataFrame
newdf = bql.combined_df(response4)


In [ ]:
newdf = newdf.drop(["DATE","EXERCISE_TYPE", 'EXPIRE_DT', 'PUT_CALL', "ORIG_IDS", 'STRIKE_PX'], axis=1)
newdf

<div style="text-align: right"><a href="#BQL-String-Interface-Basics">Return to Top ↑</a></div>

## ETF Fund Flows<a name="F"></a>
#### <font color=orange> Fixed Income Fund Flows bucketed by Fund Objective </font>

In [15]:
# Define the request string, specifying the date range for each field
request = """
let(
    #FLOW_30D = SUM(
        FUND_FLOW(Dates=range(-1m,0d),
        CURRENCY=USD,FILL=PREV)
        );
) 
get(
    sum(group(#FLOW_30D, FUND_OBJECTIVE_LONG))  as #30D_FLOW
) 
for(
    Filter(Fundsuniv(['ALL','Primary']),FUND_TYP=='ETF' AND 
                FUND_GEO_FOCUS=='U.S.' AND 
                FUND_ASSET_CLASS_FOCUS=='Fixed Income' AND 
                SUM(FUND_FLOW(Dates=range(-1m,0d),CURRENCY=USD,FILL=PREV)) > 0)
)																				
"""

# Execute the request
response5 = bq.execute(request)

In [16]:
# Display the response in a DataFrame
newdf = bql.combined_df(response5)
newdf = newdf.drop(["ORIG_IDS"], axis=1)
newdf

,DATE,CURRENCY,FUND_OBJECTIVE_LONG,#30D_FLOW
ID,,,,
Aggregate Bond,2020-06-10,USD,Aggregate Bond,1.407369e+10
Corporate,2020-06-10,USD,Corporate,3.755958e+10
Government,2020-06-10,USD,Government,4.654779e+09
Inflation Protected,2020-06-10,USD,Inflation Protected,2.057680e+09
Long Term Corporate Bond,2020-06-10,USD,Long Term Corporate Bond,2.980380e+06
Mortgage-Backed,2020-06-10,USD,Mortgage-Backed,1.316702e+09
Muni National Long,2020-06-10,USD,Muni National Long,2.153631e+07
Municipals,2020-06-10,USD,Municipals,3.571756e+09
NullGroup,2020-06-10,USD,None,1.441951e+09


## ETF Performance
#### <font color=orange> Big moves recently </font>

In [17]:
# Define the request string, specifying the date range for each field
request = """
let(
    #PctChg3day=pct_chg(dropna(px_last(start=-3d)));
) 
get(#PctChg3day) 
for(
    filter(
           fundsuniv(['PRIMARY','ACTIVE']),
           fund_asset_class_focus == 'Equity' AND 
            #PctChg3day > 10
     )
) 
with(mode='cached')														
"""

# Execute the request
response6 = bq.execute(request)

In [18]:
# Display the response in a DataFrame
newdf = bql.combined_df(response6)
newdf = newdf.drop(["CURRENCY"], axis=1)
newdf

,DATE,#PctChg3day
ID,,
BEARAKSO NO Equity,2020-06-09,21.092044
BULLGOGL NO Equity,2020-06-10,15.827339
DRIP US Equity,2020-06-09,15.047022
BEAROBX5 NO Equity,2020-06-10,11.444213
AAP2 LN Equity,2020-06-10,11.331527
6C03638 KS Equity,2020-06-10,13.573951
6C09635 KS Equity,2020-06-10,13.249145
6C12183 KS Equity,2020-06-10,12.719392
6C18611 KS Equity,2020-06-10,11.684411


<div style="text-align: right"><a href="#BQL-String-Interface-Basics">Return to Top ↑</a></div>

## Exponential Moving Avg of Skew<a name="G"></a>
#### 5 day period, 30d 90/110

In [ ]:
request = """
let(
    #Skew=implied_volatility(EXPIRY=30D,PCT_MONEYNESS=90,fill=prev)-implied_volatility(EXPIRY=30D,PCT_MONEYNESS=110,fill=prev);
    #EMAVG_Skew=emavg(close=#skew,period=5);
    #SPX_EMAVG_Skew=wavg(group(#EMAVG_Skew),group(id().weights));
    #Hist_SPX_EMAVG_Skew=rolling(#SPX_EMAVG_Skew,iterationdates=range(-2y,0d,frq=m));
) 
get(
    #Hist_SPX_emavg_skew
) 
for(
    members('spx index')
)												
"""

# Execute the request
response7 = bq.execute(request)

In [ ]:
# Display the response in a DataFrame
newdf = bql.combined_df(response7)
newdf

In [ ]:
newdf = newdf.drop(['ORIG_IDS:0', 'ORIG_IDS:1', 'ITERATION_DATE'], axis=1)
newdf

## Reading Public Portfolios<a name="H"></a>
#### e.g. using the FLNG number

In [19]:
request = """
get(
      SECURITY_DES
) 
for(
    members(['U6316781-139 Client'],Type=PORT)
)
"""

# Execute the request
response8 = bq.execute(request)


In [20]:
# Display the response in a DataFrame
newdf2 =response8[0].df()
#newdf2 = bql.combined_df(response8)
newdf2

,SECURITY_DES
ID,
AAPL UW Equity,None
BAC UN Equity,None
BRK/B UN Equity,None
C UN Equity,None
COP UN Equity,None
CSCO UW Equity,None
CVX UN Equity,None
GE UN Equity,None
GOOGL UW Equity,None


<div style="text-align: right"><a href="#BQL-String-Interface-Basics">Return to Top ↑</a></div>

## Cross Asset Query <a name="J"></a>
##### What are the companies that I am the most exposed to within the US Corporate Bond Index (`LUACTRUU`)?

In [3]:
request = """
let(
#NAME=VALUE(NAME,translatesymbols(members('LUACTRUU index'),instrumentidtype='Corp',targetIDtype='fundamentalticker'),mapby=lineage).value;

#TICKER=VALUE(FUNDAMENTAL_TICKER,translatesymbols(members('LUACTRUU index'),instrumentidtype='Corp',targetIDtype='fundamentalticker'),mapby=lineage).value;

#ID=CONCAT(CONCAT(#NAME,':  '), #TICKER);
#wgt=ID().Weights;) 

get(groupsort(SUM(GROUP(#wgt,#ID))) as #PCT) 

for(Members('LUACTRUU Index')) 

with(fill=prev)				
"""

# Execute the request
response9 = bq.execute(request)


In [22]:
# Display the response in a DataFrame
newdf3 =response9[0].df()
#newdf2 = bql.combined_df(response8)
newdf3

,ORIG_IDS,#ID,#PCT
ID,,,
JPMorgan Chase & Co: JPM US Equity,None,JPMorgan Chase & Co: JPM US Equity,2.271026
Bank of America Corp: BAC US Equity,None,Bank of America Corp: BAC US Equity,2.158335
Wells Fargo & Co: WFC US Equity,None,Wells Fargo & Co: WFC US Equity,1.889762
AT&T Inc: T US Equity,None,AT&T Inc: T US Equity,1.760113
Citigroup Inc: C US Equity,None,Citigroup Inc: C US Equity,1.717097
Goldman Sachs Group Inc/The: GS US Equity,None,Goldman Sachs Group Inc/The: GS US Equity,1.541185
Comcast Corp: CMCSA US Equity,None,Comcast Corp: CMCSA US Equity,1.482593
Morgan Stanley: MS US Equity,None,Morgan Stanley: MS US Equity,1.400931
Apple Inc: AAPL US Equity,None,Apple Inc: AAPL US Equity,1.288450


## Cross Asset Query <a name="H"></a>
##### Break out the average Equity Implied Volatility by Credit Rating for the US Corporate Index?

In [23]:
request = """
let(#buckets=if(in(BB_COMPOSITE,['BB','BB+','BB-']),'BB',
if(in(BB_COMPOSITE,['BBB','BBB+','BBB-']),'BBB',
if(in(BB_COMPOSITE,['B','B+','B-']),'B',
if(in(BB_COMPOSITE,['AA','AA+','AA-']),'AA',
if(in(BB_COMPOSITE,['AAA','AAA+','AAA-']),'AAA',
if(in(BB_COMPOSITE,['A','A+','A-']),'A',
'NR/Other'))))));

#INT_RATIO=VALUE(IMPLIED_VOLATILITY,translatesymbols(members('LUACTRUU index'),instrumentidtype='Corp',targetIDtype='fundamentalticker'),mapby=lineage).value;

#wgt=ID().Weights;

#AVG_IVOL=groupsort(WAVG(group(#INT_RATIO,#buckets),group(#wgt,#buckets)),order=desc);

#count=count(group(id,#buckets));) 

get(#AVG_IVOL,#count) 
for(Members('LUACTRUU Index'))

"""

# Execute the request
response_z = bq.execute(request)


In [24]:
# Display the response in a DataFrame
newdf4 =response_z[0].df()
#newdf2 = bql.combined_df(response8)
newdf4

,ORIG_IDS,#BUCKETS,#AVG_IVOL
ID,,,
BB,None,BB,45.772802
BBB,None,BBB,39.618565
NR/Other,None,NR/Other,39.059128
A,None,A,35.971835
AA,None,AA,30.476305
AAA,None,AAA,27.620844


<div style="text-align: right"><a href="#BQL-String-Interface-Basics">Return to Top ↑</a></div>

## SCRATCH
get( sum(group(amt_outstanding,bics_level_1_sector_name))/1000000 as#Rolling_Debt) for(filter(members('lbeatreu index'),maturity<2y)) with(currency=USD)



get(
       LOAN_TYP, MATURITY, LN_CURRENT_MARGIN().value, RTG_MOODY().value
)
for(
    members(['H319705-10 Client'],Type=PORT, Dates=2018-09-11)
)

In [ ]:
%pwd

## Example ETF Fund Flow

In [ ]:
request = """
let(#ff = FUND_FLOW(Dates=range("2019-01-01","2020-01-30"),CURRENCY="USD")/10^7;
#ff_fam = sum(dropna(group(#ff,ULT_PARENT_TICKER_EXCHANGE),true));)

get(groupsort(#ff_fam) as #FundFlowsbyFamily)

for(filter( FundsUniv(['Active','Primary']), FUND_TYP=='ETF' AND ULT_PARENT_TICKER_EXCHANGE!='STT US' and ULT_PARENT_TICKER_EXCHANGE!=NA))
      with(MODE=CACHED)

"""

# Execute the request
response_z = bq.execute(request)


In [ ]:
newdf3 =response_z[0].df()
#newdf2 = bql.combined_df(response8)
newdf3

<div style="text-align: right"><a href="#BQL-String-Interface-Basics">Return to Top ↑</a></div>